# Libs

In [1]:
import os
while os.getcwd().split('\\')[-1] != 'master-llm-rag-vnlaw':
    os.chdir('..')
    print(os.getcwd())

d:\DANG NHI\repos\master-llm-rag-vnlaw


In [ ]:
import ollama
import asyncio
from ollama import AsyncClient
import numpy as np

from langchain.vectorstores import Chroma

import chromadb
from chromadb import Settings, EmbeddingFunction, Embeddings

from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document

from nltk.tokenize import word_tokenize
import regex as re
from collections import Counter


# Load embedding

In [3]:
base_model_id = "FacebookAI/xlm-roberta-base"
trained_model_path = "./models/trained_embedding_small_data/"
data_path = "data/"

MAX_LEN = 512
OVERLAP = 50

import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
model = AutoModel.from_pretrained(trained_model_path)
model.eval()

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')
model.to(device)

print("Model ready !!!")

Model ready !!!


In [4]:
context = "Tôi đang viết trên bảng ở trường."
token = tokenizer(context, return_tensors=None, truncation=True, max_length=MAX_LEN)
tokens_text = tokenizer.convert_ids_to_tokens(token["input_ids"])
print(tokens_text)

['<s>', '▁Tôi', '▁đang', '▁viết', '▁trên', '▁bảng', '▁ở', '▁trường', '.', '</s>']


In [5]:
class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model, tokenizer, max_length=512):
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_length


    def embed_documents(self, texts):
        inputs = self.tokenizer(texts, 
                                padding='max_length', 
                                max_length=self.max_length, 
                                return_tensors="pt")
        with torch.no_grad():
            inputs = {key: value.to(device) for key, value in inputs.items()}
            embeddings = self.model(**inputs).pooler_output
    
        return embeddings.cpu().numpy().tolist()

    def embed_query(self, text):
        inputs = self.tokenizer(text, return_tensors="pt")
        with torch.no_grad():
            inputs = {key: value.to(device) for key, value in inputs.items()}
            embeddings = self.model(**inputs).pooler_output
        return embeddings.cpu().numpy().tolist()[0]

myembed = MyEmbeddingFunction(model, tokenizer)

In [6]:
# text = 'Tôi đi học'
# with torch.no_grad():
#     token = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=MAX_LEN)
#     output = model(**token)

# output = myembed.embed_documents(["Tôi đi học"])
output1 = myembed.embed_query("Tôi đang viết trên bảng ở trường")

# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# similarity = cosine_similarity(output, output1)
# print(f"Cosine similarity:\n {similarity}")

# Vector database và Embedding model

In [7]:
# client = chromadb.PersistentClient(
#     path="./data/chroma_db/",
#     settings=Settings(allow_reset=False,),)
# client.list_collections()

# collection = client.get_collection(name="VNLaws", embedding_function=myembed)
# print(collection.count())

# collection.query(
#     query_texts = ["Tôi có thể làm gì với hợp đồng lao động?"],
#     n_results=10,
#     where_document={"$contains": "Tôi có thể làm gì với hợp đồng lao động?"},
# )

In [11]:
db = Chroma(
    embedding_function=myembed, # lấy embedding model đã train được
    persist_directory='./chroma_db',
    collection_name="VNLaws",
    )
vectorstore_retriever = db.as_retriever(search_kwargs={"k": 10})

stored_data = db._collection.get()
zip_doc = list(zip(stored_data["documents"], stored_data["metadatas"]))
docs = [Document(page_content=doc, metadata=metadata) for doc,metadata in zip_doc]

In [12]:
keyword_retriever = BM25Retriever.from_documents(docs, search_kwargs={"k": 10})
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retriever, keyword_retriever], weights=[0.5, 0.5])

In [13]:
relevants = ensemble_retriever.get_relevant_documents("Tôi có thể làm gì với hợp đồng lao động trong công an nhân dân?")
relevants

[Document(metadata={'name': 'Thông tư 22/2022/TT-BCA', 'url': 'https://dulieuphapluat.vn/van-ban/lao-dong-van-ban/thong-tu-222022tt-bca-cua-bo-cong-an-ve-quy-dinh-ve-lao-dong-hop-dong-trong-cong-an-nhan-dan-1100555.html'}, page_content='Điều 13. Nội dung hợp đồng lao động\n1. Hợp đồng lao động phải có những nội dung chủ yếu theo quy định tại khoản 1 Điều 21 Bộ luật Lao động .\n2. Người lao động làm việc ở các đơn vị, bộ phận trọng yếu, cơ mật trong Công an nhân dân, trước khi ký kết hợp đồng với người lao động, đơn vị trực tiếp sử dụng phải yêu cầu người lao động cam kết bằng văn bản (theo mẫu ban hành kèm theo Thông tư này) về công tác bảo mật trong thời gian làm việc và sau khi kết thúc hợp đồng lao động; đồng thời triển khai các biện pháp phù hợp, trang bị kiến thức cần thiết cho người lao động về việc bảo vệ bí mật Nhà nước.\n3. Trong trường hợp cần quy định chi tiết hoặc để sửa đổi, bổ sung một số điều, khoản của hợp đồng lao động, người sử dụng lao động và người lao động có thể l

In [ ]:
name = [i.metadata["name"] for i in relevants]
name_counts = Counter(name)
ranked_names = sorted(name_counts.items(), key=lambda x: x[1], reverse=True)

sorted_documents = []
for name, count in ranked_names:
    for doc in relevants:
        if doc.metadata["name"] == name:
            sorted_documents.append(doc)

top_documents = sorted_documents[:10]
# sorted_doc = [doc for doc in relevants for name, _ in ranked_names if doc.metadata["name"] == name]
# while len(top_documents) < max_chunks:
#     for name, count in ranked_names:
#         doc
#         top_documents.append([doc for doc in ])

In [15]:
top_documents

[Document(metadata={'name': 'Thông tư 22/2022/TT-BCA', 'url': 'https://dulieuphapluat.vn/van-ban/lao-dong-van-ban/thong-tu-222022tt-bca-cua-bo-cong-an-ve-quy-dinh-ve-lao-dong-hop-dong-trong-cong-an-nhan-dan-1100555.html'}, page_content='Điều 13. Nội dung hợp đồng lao động\n1. Hợp đồng lao động phải có những nội dung chủ yếu theo quy định tại khoản 1 Điều 21 Bộ luật Lao động .\n2. Người lao động làm việc ở các đơn vị, bộ phận trọng yếu, cơ mật trong Công an nhân dân, trước khi ký kết hợp đồng với người lao động, đơn vị trực tiếp sử dụng phải yêu cầu người lao động cam kết bằng văn bản (theo mẫu ban hành kèm theo Thông tư này) về công tác bảo mật trong thời gian làm việc và sau khi kết thúc hợp đồng lao động; đồng thời triển khai các biện pháp phù hợp, trang bị kiến thức cần thiết cho người lao động về việc bảo vệ bí mật Nhà nước.\n3. Trong trường hợp cần quy định chi tiết hoặc để sửa đổi, bổ sung một số điều, khoản của hợp đồng lao động, người sử dụng lao động và người lao động có thể l

# Create ollama client

In [17]:
ollama_api_url = "https://87ea-35-201-213-122.ngrok-free.app/" # Thay đổi mỗi lần host

ollama_client = ollama.Client(
    host=ollama_api_url,
    headers = {'Header': 'application/json'}
)

In [ ]:
system_prompt = "Xây dựng chương trình RAG với chatbot trung thực.\n" \
"Và không bao giờ trả lời các câu hỏi liên quan đến chính trị, tôn giáo, tình dục và các vấn đề nhạy cảm khác.\n" \
"Tôi có bộ dữ liệu gồm các Lĩnh vực pháp luật Việt Nam bao gồm Bảo Hiểm, Lao Động, Nhà Đất và Tài Chính Ngân Hàng. " \
"Trong đó bao bồm các văn bản hành chính của Chính phủ.\n" \




In [45]:
# question = "Quần đảo Trường Sa thuộc tỉnh nào?"
# question = "Tôi có thể làm gì với hợp đồng lao động?"
question = "Những vấn đề liên quan đến hợp đồng lao động là gì ?"

In [47]:
def judment1_answerable(question, retry=2):
    if retry == 0:
        return None

    agent_judgment1 = 'Bạn là một agent với nhiệm vụ đánh giá câu trả lời. ' \
    'Với câu hỏi "{question}", bạn có thể trả lời câu hỏi bằng cơ sở dữ liệu của tôi được hay không (trả lời 1 nếu có thể, trả lời 0 nếu không thế). \n' \
    'Trả lời:'

    response = ollama_client.chat(
        model="gemma3:12b",
        messages=[{"role": "user", "content": system_prompt + agent_judgment1.format(question=question)}],
        stream=False,
    )
    # print("response.message.content = ", response.message.content)
    if "0" in response.message.content:
        judment1_result = 0
    elif "1" in response.message.content:
        judment1_result = 1
    else:
        return judment1_answerable(question, retry-1)
    
    return judment1_result

judment1_answerable(question, retry=2)

1

In [ ]:
# agent_search_plan = 'Bạn là một agent để dự đoán thông tin cần truy vấn.\n' \
# 'Theo bạn để trả lời câu hỏi "{question}", thì cần những thông tin gì và đến từ Lĩnh vực nào?\n' \
# 'Trả lời tối đa 3 dòng và theo cú pháp "<linh_vuc>: <thong_tin_can>", linh_vuc chỉ duy nhất một, 3 câu trả lời cho thể trùng Lĩnh vực nhưng khác nhau thông tin cần.\n' \
# 'Trả lời:\n'

# response = ollama_client.chat(
#     model="gemma3:12b",
#     messages=[{"role": "user", "content": system_prompt + agent_search_plan.format(question=question)}],
#     stream=False,
# )

# print(response.message.content)

Lao Động: Các quyền và nghĩa vụ của người lao động và người sử dụng lao động khi ký kết hợp đồng lao động.
Nhà Đất: Các quy định liên quan đến quyền sở hữu và sử dụng tài sản khi có hợp đồng lao động gắn liền với nhà ở.
Tài Chính Ngân Hàng: Các vấn đề về thanh toán, khấu trừ và các khoản phúc lợi liên quan đến hợp đồng lao động.



In [55]:
agent_search_plan = 'Bạn là một agent để dự đoán thông tin cần truy vấn.\n' \
'Theo bạn để trả lời câu hỏi "{question}", thì cần những thông tin gì ?\n' \
'Gợi ý cho tôi một câu ngắn gọn bắt đầu sau từ "Trả lời" sau đây.\n' \
'Trả lời:\n'

response = ollama_client.chat(
    model="gemma3:12b",
    messages=[{"role": "user", "content": system_prompt + agent_search_plan.format(question=question)}],
    stream=False,
)
print(response.message.content)

Trả lời: Cần tìm kiếm các văn bản hành chính, nghị định, luật liên quan đến "hợp đồng lao động", "quyền và nghĩa vụ của người lao động và người sử dụng lao động", "thời hạn hợp đồng lao động", "thủ tục chấm dứt hợp đồng lao động" trong lĩnh vực Lao Động.


In [ ]:

plans = re.sub("^Trả lời:*\s*", "", response.message.content)
plans

'Cần tìm kiếm các văn bản hành chính, nghị định, luật liên quan đến "hợp đồng lao động", "quyền và nghĩa vụ của người lao động và người sử dụng lao động", "thời hạn hợp đồng lao động", "thủ tục chấm dứt hợp đồng lao động" trong lĩnh vực Lao Động.'

In [57]:
relevants = ensemble_retriever.get_relevant_documents(plans)
relevants

[Document(metadata={'name': 'Quyết định 471/QĐ-UBND', 'url': 'https://dulieuphapluat.vn/van-ban/lao-dong-van-ban/quyet-dinh-471qd-ubnd-nam-2023-cua-uy-ban-nhan-dan-tinh-khanh-hoa-ve-viec-cong-bo-danh-muc-thu-tuc-hanh-chinh-sua-doi-bo-sung-linh-vuc-quan-ly-lao-dong-ngoai-nuoc-thuoc-tham-quyen-giai-quyet-cua-so-lao-dong-thuong-binh-va-xa-hoi-ban-quan-ly-khu-kinh-te-van-phong-tinh-khanh-hoa-1101732.html'}, page_content='ở nước ngoài theo hợp đồng; - Quyết định số 58/QĐ-LĐTBXH ngày 26/01/2022 của Bộ trưởng Bộ Lao động - Thương binh và Xã hội về việc công bố các thủ tục hành chính mới ban hành, sửa đổi, bổ sung, bãi bỏ lĩnh vực quản lý lao động ngoài nước thuộc phạm vi chức năng quản lý nhà nước của Bộ Lao động - Thương binh và Xã hội.'),
 Document(metadata={'name': 'Hướng dẫn 33/HD-VKSTC', 'url': 'https://dulieuphapluat.vn/van-ban/lao-dong-van-ban/huong-dan-33hd-vkstc-cua-vien-kiem-sat-nhan-dan-toi-cao-ve-mot-so-noi-dung-co-ban-trong-cong-tac-kiem-sat-viec-giai-quyet-vu-an-tranh-chap-lao-d

In [ ]:
name = [i.metadata["name"] for i in relevants]
name_counts = Counter(name)
ranked_names = sorted(name_counts.items(), key=lambda x: x[1], reverse=True)

sorted_documents = []
for name, count in ranked_names:
    for doc in relevants:
        if doc.metadata["name"] == name:
            sorted_documents.append(doc)

top_documents = sorted_documents[:10]
top_documents


[Document(metadata={'name': 'Thông tư 22/2022/TT-BCA', 'url': 'https://dulieuphapluat.vn/van-ban/lao-dong-van-ban/thong-tu-222022tt-bca-cua-bo-cong-an-ve-quy-dinh-ve-lao-dong-hop-dong-trong-cong-an-nhan-dan-1100555.html'}, page_content='Điều 23. Lập, cập nhật, quản lý và sử dụng sổ quản lý lao động\n1. Việc lập, cập nhật, quản lý và sử dụng sổ quản lý lao động thực hiện theo Nghị định số 145/2020/NĐ-CP ngày 14 tháng 12 năm 2020 của Chính phủ quy định chi tiết và hướng dẫn thi hành một số điều của Bộ luật Lao động về điều kiện lao động và quan hệ lao động.\n2. Sổ quản lý lao động được bảo quản theo chế độ quản lý hồ sơ cán bộ.\nĐiều 24. Kỷ luật lao động, trách nhiệm vật chất\n1. Nội quy lao động\nNgười sử dụng lao động phải ban hành nội quy lao động, nếu sử dụng từ 10 người lao động trở lên thì nội quy lao động phải bằng văn bản, nếu sử dụng dưới 10 người lao động thì không bắt buộc ban hành nội quy lao động bằng văn bản nhưng phải thỏa thuận nội dung về kỷ luật lao động, trách nhiệm vậ

In [ ]:
reference = np.unique([(doc.metadata["url"]) for doc in top_documents])

In [65]:
answer_prompt_template = \
    """
    Bạn là một chuyên gia trả lời câu hỏi về luật về các lĩnh vực bao gồm Bảo Hiểm, Lao Động, Nhà Đất và Tài Chính Ngân Hàng và câu trả lời dựa trên thông tin được cung cấp. \n
    Hãy trả lời câu hỏi ngắn gọn và dễ hiểu. Nếu không có thông tin để trả lời câu hỏi, hãy gợi ý câu hỏi khác phù hợp hơn. \n
    Đây là một số thông tin bạn có được:\n
    --------\n
    {context} \n
    --------\n
    Dựa vào thông tin trên hãy trả lời câu hỏi sau đây: {question} \n
    Trả lời: 
    """

context="\n--------\n".join([doc.page_content for doc in top_documents])

answer_prompt = answer_prompt_template.format(context=context, question=question)
response = ollama_client.chat(
    model="gemma3:12b",
    messages=[{"role": "user", "content": system_prompt + answer_prompt}],
    stream=False,
)
print(response.message.content)

Dựa vào thông tin được cung cấp, những vấn đề liên quan đến hợp đồng lao động bao gồm:

*   **Hợp đồng lao động sau thời gian nhất định:** Trường hợp người lao động làm việc ở nước ngoài theo hợp đồng.
*   **Quy trình và thủ tục:** Các thủ tục hành chính mới ban hành, sửa đổi, bổ sung, bãi bỏ liên quan đến quản lý lao động ngoài nước.
*   **Giải quyết tranh chấp:**  Khi có phát sinh, vướng mắc trong quá trình thực hiện, cần phản hồi để sửa đổi, bổ sung quy chế.
*   **Các vấn đề liên quan đến hợp đồng lao động theo thời gian:** Việc hợp đồng lao động có thời hạn hoặc không có thời hạn cần được quy định rõ ràng để đảm bảo quyền lợi cho cả hai bên.
*   **Những vấn đề liên quan đến hợp đồng lao động trong khu vực doanh nghiệp nhà nước:** Cần có chính sách đãi ngộ đủ hấp dẫn để giữ chân người lao động.
*   **Quyền lợi và nghĩa vụ:** Cần có công tác thông tin, truyền thông để người lao động hiểu rõ quyền lợi, nghĩa vụ của mình trong hợp đồng lao động.
